In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
text = 'I love learning deep learning with pytorch. Pytorch makes machine learning fun.'

words = text.lower().split()

vocab = sorted(set(words))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

vocab_size = len(vocab)

print("Vocabulary: ", vocab)
print("Vocab Size: ", vocab_size)

Vocabulary:  ['deep', 'fun.', 'i', 'learning', 'love', 'machine', 'makes', 'pytorch', 'pytorch.', 'with']
Vocab Size:  10


In [4]:
X = []
y = []

for i in range(len(words) - 1):
    X.append(word2idx[words[i]])
    y.append(word2idx[words[i+1]])

X = torch.tensor(X)
y = torch.tensor(y)

print("X: ", X)
print("y: ", y)

X:  tensor([2, 4, 3, 0, 3, 9, 8, 7, 6, 5, 3])
y:  tensor([4, 3, 0, 3, 9, 8, 7, 6, 5, 3, 1])


In [ ]:
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(NextWordPredictor, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
     
    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out

In [6]:
embed_size = 10
hidden_size = 20

model = NextWordPredictor(vocab_size, embed_size, hidden_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
num_epochs = 200

for epoch in range(num_epochs):
    total_loss = 0
    
    for i in range(len(X)):
        input_word = X[i].unsqueeze(0).unsqueeze(0)
        target_word = y[i].unsqueeze(0)

        output = model(input_word)
        loss = criterion(output, target_word)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

Epoch [50/200], Loss: 3.4732
Epoch [100/200], Loss: 3.4203
Epoch [150/200], Loss: 3.4021
Epoch [200/200], Loss: 3.3893


In [10]:
def predict_next_word(model, word, top_k=3):
    model.eval()
    idx = word2idx.get(word.lower(), None)
    if idx is None:
        return "Word not in vocabulary."
    
    input_word = torch.tensor([[idx]])  # shape: (1,1)
    with torch.no_grad():
        output = model(input_word)
        probs = torch.softmax(output, dim=1)
        top_idx = torch.topk(probs, top_k).indices.squeeze().tolist()
    
    predictions = [idx2word[i] for i in top_idx]
    return predictions

In [11]:
print(predict_next_word(model, "pytorch"))
print(predict_next_word(model, "learning"))
print(predict_next_word(model, "i"))

['makes', 'pytorch', 'learning']
['with', 'deep', 'fun.']
['love', 'learning', 'pytorch.']
